In [ ]:
import os
import msvcrt
import re

import cv2
import numpy as np
from colorama import Fore, Style

In [ ]:
def standard_filename(string):
    pattern = r"[^a-zA-Z0-9\u4E00-\u9FA5_]+"
    return re.sub(pattern, "_", string)

In [ ]:
def cinput(prompt: str) -> str:
    while msvcrt.kbhit():
        msvcrt.getch()
    return input(prompt)

In [ ]:
class CS:

    @staticmethod
    def red(string: str) -> str:
        return Fore.RED + string + Style.RESET_ALL

    @staticmethod
    def green(string: str) -> str:
        return Fore.GREEN + string + Style.RESET_ALL

    @staticmethod
    def purple(string: str) -> str:
        return Fore.MAGENTA + string + Style.RESET_ALL

    @staticmethod
    def yellow(string: str) -> str:
        return Fore.YELLOW + string + Style.RESET_ALL

    @staticmethod
    def blue(string: str) -> str:
        return Fore.LIGHTBLUE_EX + string + Style.RESET_ALL

In [ ]:
def compress_image(
    input_path: str,
    output_path: str,
    target_max_size: int = 50 * 1024,
    min_quality: int = 10,
    min_scale: float = 0.2,
) -> bool:
    """
    智能压缩图片到指定大小以下, 并统一转换为JPG格式

    Args:
        input_path (str): 输入图片路径
        output_path (str): 输出图片路径
        target_max_size (int): 目标最大大小, 单位为字节
        min_quality (int): 最小压缩质量, 取值范围 0-100
        min_scale (float): 最小缩放比例, 取值范围 0-1

    Returns:
        bool: 是否成功压缩
    """
    try:
        with open(input_path, "rb") as f:
            image_data = np.frombuffer(f.read(), np.uint8)
            image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
    except Exception as e:
        print(f"读取图片失败: {e}")
        return False

    # 如果原始图片已经小于目标大小, 直接保存为JPG
    original_size = os.path.getsize(input_path)
    if original_size <= target_max_size:
        try:
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 95]
            _, buffer = cv2.imencode(".jpg", image, encode_param)
            with open(output_path, "wb") as f:
                f.write(buffer)
            return True
        except Exception as e:
            print(f"保存图片失败: {e}")
            return False

    # 先只压缩质量, 不缩放尺寸
    quality_low, quality_high = min_quality, 95
    result_buffer = None
    while quality_low <= quality_high:
        quality_mid = (quality_low + quality_high) // 2
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality_mid]
        _, buffer = cv2.imencode(".jpg", image, encode_param)
        size = len(buffer)
        if size <= target_max_size:
            result_buffer = buffer
            quality_low = quality_mid + 1
        else:
            quality_high = quality_mid - 1

    # 如果只靠质量压缩就能满足, 直接保存
    if result_buffer is not None:
        with open(output_path, "wb") as f:
            f.write(result_buffer)
        return True

    # 需要缩放尺寸
    scale = 0.9
    while scale >= min_scale:
        new_w = int(image.shape[1] * scale)
        new_h = int(image.shape[0] * scale)
        resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        # 再用二分法找合适质量
        quality_low, quality_high = min_quality, 95
        result_buffer = None
        while quality_low <= quality_high:
            quality_mid = (quality_low + quality_high) // 2
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality_mid]
            _, buffer = cv2.imencode(".jpg", resized, encode_param)
            size = len(buffer)
            if size <= target_max_size:
                result_buffer = buffer
                quality_low = quality_mid + 1
            else:
                quality_high = quality_mid - 1
        if result_buffer is not None:
            with open(output_path, "wb") as f:
                f.write(result_buffer)
            return True
        scale *= 0.8  # 继续缩小尺寸

    print(
        f"无法将 {os.path.basename(input_path)} 压缩到目标范围, 最终大小: {os.path.getsize(output_path)/1024:.2f} KB"
    )
    return False